<header>
   <p  style='font-size:36px;font-family:Arial;color:#F0F0F0; background-color: #00233c; padding-left: 20pt; padding-top: 20pt;padding-bottom: 10pt; padding-right: 20pt;'>
       Anomaly Detection in Robot Welding Process
  <br>
       <img id="teradata-logo" src="https://storage.googleapis.com/clearscape_analytics_demo_data/DEMO_Logo/teradata.svg" alt="Teradata" style="width: 125px; height: auto; margin-top: 20pt;">
    </p>
</header>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Introduction</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Detecting anomalies reduces issues and delays in many industries, especially in the manufacturing field. There have been approaches to detect anomalies in the past, such as engineering rules and graph and deep learning. However, it still proves difficult to detect all the existing anomalies. Plus, companies are striving to minimize false positives, cope with the diversity of sensors and metrology issues, and deliver actionable insights at a business pace. Fortunately, Teradata and ClearScape Analytics have the solution. In ClearScape Analytics, users can execute all steps of anomaly detection from data preparation and exploration to model training and evaluations and adjustments. These analyses can improve the process and ensure accuracy in anomaly detection.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Spot Welding Quality Assessment</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Spot welding is a common technique used for welding car body panels, particularly in the assembly of smaller parts and components. Spot welding involves using a pair of copper electrodes to apply a series of short, high-current welding pulses to the metal, fusing the parts together at specific points or “spots”.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The automotive industry is known for its high level of automation, and spot welding is one of the most automated processes, heavily reliant on robots to improve efficiency, reduce labor costs, and improve the consistency and quality of the finished product. Poor welding quality is rare, but even so, the consequences of poor quality may not be negligible in terms of rework costs and customer satisfaction, especially when quality issues are detected too late.</p>

<img  src="images/AnomalyWelding.png"/>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Spot welding is a resistance welding process that uses large electrical current. There are many ways to assess the quality of a spot, like tensile or ultrasonic testing to assess the weld strength or the analysis of the welding current measured and recorded during the welding process. In this demo, we focus on the analysis of the anomalies in the welding spot due to welding current, and more specifically the resistance, i.e. the voltage-current ratio which impacts the quality of the welding. The shape of the resistance curve depends on many factors like  the nature of the materials, the geometry, and the quality of the electrodes etc. </p>


<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Business Values</b></p>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Improve accuracy in the production and manufacturing process.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Reduce the number of false positive anomalies detected in a system.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Decrease additional costs and time wasted due to undetected anomalies.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Determine patterns and significant factors that lead to anomalies.</li></p>
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Why Vantage?</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Many organizations fail to realize value from their ML and AI investments due to a lack of scale. It is estimated that for broad adoption across many industries, the number of models and model deployments needs to scale 100-1000x larger than their organizations currently support.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The unique massively-parallel architecture of Teradata Vantage allows users to prepare data, train, evaluate, and deploy models at unprecedented scale.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this particular use case, the volume of machine sensor data was so great that millions of ML models were created to derive analytic features that ultimately deployed tens of thousands of models for real-time scoring. This extent of scale is only possible by combining the power of Vantage with native ClearScape Analytic functions.</p>


</header>

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>1. Connect to Vantage.</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the section, we import the required libraries and set environment variables and environment paths (if required).</p>

In [ ]:
%%capture
# # '%%capture' suppresses the display of installation steps of the following packages
# !pip install tdsense
# !pip install imblearn
# !pip install xgboost==1.7.3
# !pip install colorlover
# !pip install teradataml --upgrade teradataml

<div class="alert alert-block alert-info">
<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Note: </b><i>The above statements may need to be uncommented if you run the notebooks on a platform other than ClearScape Analytics Experience that does not have the libraries installed. If you uncomment those installs, be sure to restart the kernel after executing those lines to bring the installed libraries into memory. The simplest way to restart the Kernel is by typing zero zero: <b> 0 0</b></i></p>
</div>

In [ ]:
import json
import getpass
import pandas as pd
import datetime
from teradataml import *

# import tdsense
# from tdsense.plot import plotcurves

import numpy as np # linear algebra
import matplotlib.pyplot as plt
import sklearn
from sklearn import preprocessing
# from tdsense.clustering import hierarchy_dendrogram, hierarchy_clustering
# from tdnpathviz.visualizations import plotcurves
%matplotlib inline

from sklearn import datasets
from sklearn2pmml.pipeline import PMMLPipeline
from sklearn2pmml import sklearn2pmml
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, roc_auc_score, f1_score,confusion_matrix, roc_curve, ConfusionMatrixDisplay
import time
import pytz


import os
from jdk4py import JAVA, JAVA_HOME, JAVA_VERSION
# Set java path

os.environ['PATH'] = os.environ['PATH'] + os.pathsep + str(JAVA_HOME)
os.environ['PATH'] = os.environ['PATH'] + os.pathsep + str(JAVA)[:-5]

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from collections import defaultdict
import plotly.offline as offline
offline.init_notebook_mode()


from teradataml.dataframe.sql_functions import case
from teradataml import db_drop_table
configure.byom_install_location = "mldb"

display.max_rows = 5
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=DeprecationWarning)
warnings.simplefilter(action='ignore', category=RuntimeWarning)
warnings.simplefilter(action='ignore', category=FutureWarning)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will be prompted to provide the password. We will enter the password, press the Enter key, and then use the down arrow to go to the next cell. Begin running steps with Shift + Enter keys.</p>

In [ ]:
%run -i ../startup.ipynb
eng = create_context(host = 'host.docker.internal', username='demo_user', password = password)
print(eng)

In [ ]:
%%capture
execute_sql('''SET query_band='DEMO=AnomalyDetection.ipynb;' UPDATE FOR SESSION; ''')

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>2. Getting Data for This Demo</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have provided data for this demo on cloud storage. We have the option of either running the demo using foreign tables to access the data without using any storage on our environment or downloading the data to local storage, which may yield somewhat faster execution. However, we need to consider available storage. There are two statements in the following cell, and one is commented out. We may switch which mode we choose by changing the comment string.</p>   


In [ ]:
# %run -i ../run_procedure.py "call get_data('DEMO_AnomalyDetection_cloud');"
 # Takes about 50 seconds
%run -i ../run_procedure.py "call get_data('DEMO_AnomalyDetection_local');"
 # Takes about 2 minute 30 secs

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Optional step – We should execute the below step only if we want to see the status of databases/tables created and space used.</p>

In [ ]:
%run -i ../run_procedure.py "call space_report();"

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>3. Analyze the raw data set</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let us start by creating a teradataml dataframe. A "Virtual DataFrame" that points directly to the dataset in Vantage.</p>



In [ ]:
Sensor_Data = DataFrame(in_schema('DEMO_AnomolyDetection', 'Sensor_Data'))
Sensor_Data

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We get the above data from sensors. We focus on one plant (PLANT=1) and one robot (ROBOT_ID=41). The Partition_ID is the type of welding, ID is the WELDING_ID, X is time required for welding in ms and Y is the RESISTANCE. We create a view with the columns required to get data with proper column names.</p>

In [ ]:
%%capture
query = f"""
REPLACE VIEW DEMO_AnomolyDetection.V_dataset_01 AS
SELECT
    1 AS PLANT
,   {41} AS ROBOT_ID
,   CAST(A.PARTITION_ID AS BIGINT) AS WELDING_TYPE
,   CAST((DATE '{str(datetime.now()).split(' ')[0]}'  + FLOOR((WELDING_ID-700*WELDING_TYPE)/100))  AS DATE FORMAT 'YYYY-MM-DD') AS WELDING_DAY
,   CAST(A.ID AS BIGINT) AS WELDING_ID
,   CAST(A.X AS INTEGER) AS TIME_MS
,   A.Y AS RESISTANCE
FROM DEMO_AnomolyDetection.Sensor_Data A
"""
execute_sql(query)

In [ ]:
welding_dataset_new = DataFrame(in_schema('DEMO_AnomolyDetection', 'V_dataset_01'))
welding_dataset_new

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.1 - Some aggregations and visualization. </b></p>


<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will check the histogram based on the minimum and maximum Time for welding.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A histogram is a better way to assess distribution, to cope with the scalability, it is recommended to compute the histogram bins in-database to leverage the Massively Parallel Architecture of Teradata Vantage. For that, we use the Histogram function of teradataml that pushes down the computations to Vantage.</p>

In [ ]:
welding_duration_ms = welding_dataset_new. \
                        groupby(['PLANT','ROBOT_ID','WELDING_TYPE', 'WELDING_ID']). \
                        agg({'TIME_MS':['min','max','count']})
welding_duration_ms

In [ ]:
from teradataml import Histogram
obj = Histogram(data=welding_duration_ms,
                    target_columns="count_TIME_MS",
                    method_type="Scott")
res = obj.result.sort('MinValue')
res

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We can see that we have calculated the histogram values using the teradataml functions. Clearscape Analytics can easily integrate with 3rd party visualization tools like Tableau, PowerBI or many python modules available like plotly, seaborn etc. We can do all the calculations and pre-processing on Vantage and pass only the necessary information to visualization tools, this will not only make the calculation faster but also reduce the time due to less data movement between tools. We do the data transfer for this and the subsequent visualizations wherever necessary.</p>

In [ ]:
res = obj.result.sort('MinValue').to_pandas()
res['duration_ms'] = [str(row['MinValue'])+'-'+str(row['MaxValue']) for i,row in res.iterrows()]
res.plot(x='duration_ms',y='CountOfValues',kind='bar', figsize=(15,10), legend=False,xlabel='Duration(ms)', ylabel='Welding Counts')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In the above histogram we can see the bins between the Min and the Max value of the durations and the welding counts.</p> 
<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>3.2 - More advanced processing using window functions and delta_t </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Resistance is an important parameter in resistance welding. The resistance should not vary too much. If there are any significant changes in resistance over time, it could indicate an issue with the weld quality. For example, an unusually high resistance could indicate poor contact between the parts being welded or a problem with the welding equipment.</p>

In [ ]:
welding_dataset_new.loc[welding_dataset_new.WELDING_ID == 854]

In [ ]:
from tdnpathviz.visualizations import plotcurves
plotcurves(welding_dataset_new.loc[welding_dataset_new.WELDING_ID == 854],field='RESISTANCE',row_axis='TIME_MS', series_id='WELDING_ID',select_id=None)

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above graph shows the variation of the resistance of the welding with respect to time. We see that the most interesting part lies between 40 and 400ms from the start of the curve.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next we apply the window function on the resistance to smooth the resistance and taking the mean value.</p>


In [ ]:
# curve smoothing
window_for_smoothing = welding_dataset_new.RESISTANCE.window(
                            partition_columns   = "WELDING_ID",
                            order_columns       = 'TIME_MS',
                            window_start_point  = -15,
                            window_end_point    = 15
)
welding_dataset_smooth = welding_dataset_new.assign(RESISTANCE_SMOOTHED = window_for_smoothing.mean())

In [ ]:
id_curve = 854
single_welding = welding_dataset_smooth[welding_dataset_smooth.WELDING_ID == id_curve].sort('TIME_MS')
single_welding

In [ ]:
figure = Figure(width=1000, height=400, image_type="jpg",
                        heading="RESISTANCE and RESISTANCE SMOOTHED")
plot = single_welding.plot(x=single_welding.TIME_MS, y=[single_welding.RESISTANCE, single_welding.RESISTANCE_SMOOTHED],
                    style=['blue', 'red'],xlabel='time in ms', ylabel='resistance ',figure=figure)
plot.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above graph shows the variation of the resistance of the welding with respect to time and the smoothed resistance, as shown by the Red line, after applying the window function.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The window function generates a Window object on a teradataml DataFrame Column to run window aggregate functions.
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Function allows user to specify window for different types of computations:
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Cumulative
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Group
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Moving
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>Remaining
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>By default, window with Unbounded Preceding and Unbounded following is considered for calculation.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Next we calculate the derivative by using the lead function and taking the difference of the lead value and the mean value of the resistance. Applying a window function to smooth the resistance curve helps to eliminate noise and makes it easier to see the overall trend. The derivative of the resistance gives an indication of how quickly the resistance is changing, which can be a useful measure for detecting anomalies and predicting potential issues.</p>


In [ ]:
# let's compute the lead
window_for_lead = welding_dataset_smooth.RESISTANCE_SMOOTHED.window(
                            partition_columns   = "WELDING_ID",
                            order_columns       = 'TIME_MS')

In [ ]:
welding_dataset_smooth = welding_dataset_smooth.assign(RESISTANCE_SMOOTHED_AFTER = window_for_lead.lead())
welding_dataset_smooth = welding_dataset_smooth.assign(DERIVATIVE = (welding_dataset_smooth.RESISTANCE_SMOOTHED_AFTER - welding_dataset_smooth.RESISTANCE_SMOOTHED).zeroifnull())
welding_dataset_smooth.sort(['WELDING_ID','TIME_MS'])

In [ ]:
id_curve = 854
single_welding_subplot = welding_dataset_smooth[welding_dataset_smooth.WELDING_ID == id_curve].sort('TIME_MS')
single_welding_subplot

In [ ]:
from teradataml import subplots
# fig, axes = subplots(grid = {(1, 1): (1, 1),(2, 1): (1, 2)})
# Plot 1980 data at first Axis.
fig, axes = subplots(nrows=2, ncols=1)
plot = single_welding_subplot.plot(x=single_welding_subplot.TIME_MS, 
                    y=[single_welding_subplot.RESISTANCE, single_welding_subplot.RESISTANCE_SMOOTHED],
                    legend=["RESISTANCE", "RESISTANCE SMOOTHED"],
                    figure=fig,
                    style=['blue', 'red'],xlabel='time in ms', ylabel='resistance ',               
                    ax=axes[0])

# Plot 1981 data at second Axis.
plot = single_welding_subplot.plot(x=single_welding_subplot.TIME_MS, 
                    y=single_welding_subplot.DERIVATIVE,
                    legend=["DERIVATIVE"],
                    figure=fig,
                    style="red",xlabel='time in ms', ylabel='derivative ' ,              
                    ax=axes[1])
plot.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We see that the most interesting part lies between 40 and 400ms from the start of the curve, so we plot only that subset.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>It is hard to assess the diversity of curve shapes in this plot since many of them are superimposed. However, we see in the middle of the picture a sharp drop that looks unusual. Moreover, we guess that there are shifts in time and height.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>4. Feature Engineering</b></p>

In [ ]:
welding_dataset_new.columns

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will create a feature table by using different functions on the Resistance column. Valid values for functions are: 'count', 'sum', 'min', 'max', 'mean', 'std', 'percentile', 'unique','median', 'var', 'skew', 'kurtosis'. </p>

In [ ]:
features = welding_dataset_new.loc[welding_dataset_new.TIME_MS > 20,:]. \
        groupby(welding_dataset_new.columns[0:5]). \
        agg({
            'TIME_MS':['min','max'],
            'RESISTANCE':['count', 'sum', 'min', 'max', 'mean', 'std', 'percentile', 'unique','median', 'var','skew','kurtosis']
        })
features

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>5. Anomaly Detection on Sensor Data</b></p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let's start by getting the feature columns from the features tables</p>   

In [ ]:
feature_names = features.columns[7::]
feature_names

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.1 Clustering by curve shape</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>To cluster time series by shapes, we will use the Dynamic Time Warping (DTW) distance that measures the similarity between two time series. This distance is well adapted to this kind of problem since it provides robustness to shifts in time and height.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Distance Matrix in-database Computations</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The ClearScape Analytics DTW function computes at scale distances between one reference curve to a set of curves, a many-to-one approach. ClearScape Analytics offers in database dynamic time warping function, callable in SQL as TD_DTW. TD_DTW measures the similarity of two time series. The Dynamics Time Warping (DTW) algorithm is used for space and time. The algorithm uses the FastDTW algorithm. TD_DTW measures the similarity of two time series. The Dynamics Time Warping (DTW) algorithm is used for space and time. The algorithm uses the FastDTW algorithm. This function computes at scale the DTW distances between one reference curve to a set of curves, a many-to-one approach. We want to compute the distance matrix of our subset, i.e. the DTW distance between each curve. The distance matrix is symmetric, since the DTW is, hence we only need to compute the triangular matrix. We wrapped this computation in the tdsense package that calls the TD_DTW function and iterates on the matrix row to compute and store the whole triangular distance matrix in a table.</p>

In [ ]:
overview = welding_dataset_new.groupby('WELDING_DAY').count(distinct=True)
dates = list(overview.to_pandas().reset_index()['WELDING_DAY'].values.astype('str'))
dates

In [ ]:
subset = welding_dataset_new[ \
                 (welding_dataset_new['PLANT'] == 1) & \
                 (welding_dataset_new['ROBOT_ID'] == 41) & \
                 (welding_dataset_new['WELDING_TYPE'] in (8,9)) & \
                 (welding_dataset_new['WELDING_DAY'].isin(dates)) \
                ]

In [ ]:
subset_zoom = subset[(subset.TIME_MS < 400) & (subset.TIME_MS > 40)]
subset_zoom.shape

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The subset of data we have taken contains 7 columns and 344,622 rows.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Since this is a 2CPU system, the below computation takes around more than 2 hours for 350k rows and so we have pre calculated it and stored in the table in database.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'><i>**In case we still want to compute the matrix please set the If part of the below code to <b>True</b> instead of <b>False</b></i></p>

In [ ]:
if False:
    dtw_matrix = dtw_distance_matrix_computation2(subset_zoom,field='RESISTANCE',
                                     table_name=dtw_result_table,
                                     schema_name = Param['database'],
                                     row_axis='TIME_MS',
                                     series_id = 'WELDING_ID')
else:
    dtw_matrix = DataFrame(in_schema('DEMO_AnomolyDetection','DTW_Matrix'))

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.2 Hierarchical clustering with Scipy</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now the distance matrix is available, we can perform the clustering. Here, we will use the open-source package Scipy and its cluster.hierarchy modules, that have been used in a tdsense for convenience.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Hierarchical clustering is an alternative class of clustering algorithms that produce 1 to n clusters, where n is the number of observations in the data set. As you go down the hierarchy from 1 cluster (contains all the data) to n clusters (each observation is its own cluster), the clusters become more and more similar (almost always).</p>

In [ ]:
dtw_matrix_loc = dtw_matrix.sort(columns=['WELDING_ID_2','WELDING_ID_1']).to_pandas(all_rows=True)
dtw_matrix_loc

In [ ]:
from tdsense.clustering import hierarchy_dendrogram, hierarchy_clustering
linked, labelList = hierarchy_dendrogram(dtw_matrix_loc, cluster_distance = 'ward')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The dendrogram is useful for visualizing the structure of the hierarchical clustering and identifying the optimal number of clusters to use for further analysis. The optimal number of clusters can be determined by examining the dendrogram to identify a level at which the clusters start to merge more slowly or by using a threshold for the maximum distance between clusters.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The resulting dendrogram as above shows how the hierarchical clustering algorithm has merged the data points into clusters based on their pairwise distances using the Ward linkage criterion. The dendrogram is a summary of the distance matrix. The X axis has the WELDING_ID but not visible as we have more than 450k rows. Looking at the dendrogram, we see that we have about 6 clusters. When selected 6, here is what we have got.</p>

In [ ]:
cluster = hierarchy_clustering(linked, labelList, n_clusters=6)
cluster.head()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above dendogram is for only 6 clusters with the colors representing the different clusters. Now, we plot the Resistance curves for each cluster.</p>

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(20,10))
colors = cluster[['cluster','leaves_color_list']].copy().drop_duplicates()
for k in range(6):
    plt.subplot(2,3,k+1)
    img = plotcurves( subset_zoom,
                      field='RESISTANCE',
                      row_axis='TIME_MS',
                      series_id='WELDING_ID',
                      select_id=list(cluster[cluster.cluster ==k].CURVE_ID.values),
                      noplot=True)
    plt.imshow(img)
    plt.title('cluster : ' +str(k) + '\n' + str(cluster.groupby('cluster').count()['CURVE_ID'][k]) + ' obs.',fontdict = {'fontsize' : 10, 'color':colors.leaves_color_list.values[k]})
    plt.axis('off')

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>And if we plot the curves per cluster, we spot the curves with a sharp drop(cluster 4) and these are the curves we are interested in, i.e. the curve exhibiting the anomaly we are looking for. We note also the other clusters are looking more or less similar. By monitoring the resistance over time and calculating its derivative, you can detect any sudden changes or anomalies. Anomalies might indicate a problem with the welding process, such as a sudden drop in current or a sudden increase in resistance. </p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.3 Create the anomaly dataset</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Now we create a table containing the anomaly flag that will be the target of a supervised machine learning model or a relevant KPI to monitor in production dashboards.</p>



In [ ]:
target = cluster.copy().drop('leaves_color_list',axis=1)
target = target[target.cluster.isin([1,2])]
target['WELDING_ID'] = target['CURVE_ID']
target['anomaly'] = 0
target.loc[target.cluster==2,'anomaly'] = 1
target.drop(['cluster','CURVE_ID'],axis=1, inplace=True)
target.groupby('anomaly').count().plot(y='WELDING_ID',kind='bar',figsize=(10,10))
copy_to_sql( target,
                  table_name = 'Anomoly_Target',
                  if_exists='replace',
                  primary_index='WELDING_ID')

In [ ]:
anomalies = DataFrame('Anomoly_Target')
anomalies

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The above anomaly data has the welding ID and the anomaly flag.</p>
<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>5.4 Build the analytical dataset </b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We prepare the analytical dataset by joining the feature table with the anomaly table using the Welding ID so that we get the anomalies for the weldings.</p>

In [ ]:
ADS = features[['WELDING_ID']+feature_names].join(other=anomalies, how='inner', on='WELDING_ID=WELDING_ID',rsuffix='r',lsuffix='l')
ADS = ADS.assign(WELDING_ID=ADS.l_WELDING_ID).drop(['l_WELDING_ID','r_WELDING_ID'],axis=1).select(['WELDING_ID']+feature_names+['anomaly'])
ADS

In [ ]:
ADS.shape

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The analytical dataset we created has 14 columns and 391 rows which will be used to build the model below.</p>

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>6. Build the model </b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have datasets in which different columns have different units – like one column can be in kilograms, while another column can be in centimetres. If we feed these features to the model as is, there is every chance that one feature will influence the result more due to its value than the others. But this doesn’t necessarily mean it is more important as a predictor. So, to give importance to all the features we need feature scaling.</p>
    
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Here, we apply the Standard scale and transform functions which are ScaleFit and ScaleTransform functions in Vantage. ScaleFit() function outputs statistics to input to ScaleTransform() function, which scales specified input DataFrame columns.</p> 

In [ ]:
from teradataml import ScaleFit , ScaleTransform
scaler = ScaleFit(
                    data=ADS,
                    target_columns=feature_names,
                    scale_method="STD",
                    global_scale=False)

In [ ]:
ADS_scaled = ScaleTransform(data=ADS,
                         object=scaler.output,
                         accumulate="anomaly").result
ADS_scaled

In [ ]:
df = ADS_scaled.to_pandas()

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.1 Create a model file using the python libraries.</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The Vantage Bring Your Own Model (BYOM) package gives data scientists and analysts the ability to operationalize predictive models in Vantage. Predictive models trained in external tools with sample data can be used to score data stored in Vantage using the BYOM Predict. Create or convert your predictive model using a supported model interchange format (PMML, MOJO, ONNX, Dataiku, and DataRobot are currently available), store it in a Vantage table, and use the BYOM PMMLPredict, H2OPredict, ONNXPredict, DataikuPredict, or DataRobotPredict to score your data with the model.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>A problem with imbalanced classification is that there are too few examples of the minority class for a model to effectively learn the decision boundary. One way to solve this problem is to oversample the examples in the minority class. the most widely used approach to synthesizing new examples is called the Synthetic Minority Oversampling Technique, or SMOTE for short. SMOTE works by selecting examples that are close in the feature space, drawing a line between the examples in the feature space and drawing a new sample at a point along that line.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Then we use the RandomForestClassifier to create the model. A random forest is a meta estimator that fits a number of decision tree classifiers on various sub-samples of the dataset and uses averaging to improve the predictive accuracy and control over-fitting. The Random forest classifier creates a set of decision trees from a randomly selected subset of the training set. It is basically a set of decision trees (DT) from a randomly selected subset of the training set and then It collects the votes from different decision trees to decide the final prediction.</p>

In [ ]:
X_train = df[feature_names]
y_train = df['anomaly']

In [ ]:
# Balance the training set using SMOTE
smote = SMOTE(random_state=42)
X_train, y_train = smote.fit_resample(X_train, y_train)


# Create a random forest classifier
model = RandomForestClassifier(n_estimators=10,max_depth= 3, random_state=42)

# Create a pipeline that includes the SMOTE transformer and the model
pipeline = PMMLPipeline([ ('model', model)])


In [ ]:
# Train the pipeline
start = time.time()
pipeline.fit(X_train, y_train)
end = time.time()
print('duration : ', end-start, 's')

In [ ]:
# make predictions on the training set
y_train_pred = pipeline.predict(X_train)

# calculate and print the accuracy score
acc = accuracy_score(y_train, y_train_pred)
print("Accuracy: {:.2f}%".format(acc * 100))

# calculate and print precision, AUC and F1-score
prec = precision_score(y_train, y_train_pred)
print("Precision: {:.2f}%".format(prec * 100))

# calculate AUC, AUC requires probability for positive class
prob = pipeline.predict_proba(X_train)[:, 1]
auc = roc_auc_score(y_train, prob)
print("AUC: {:.2f}%".format(auc * 100))

f1 = f1_score(y_train, y_train_pred)
print("F1-Score: {:.2f}%".format(f1 * 100))

In [ ]:
pmml_metrics=pd.DataFrame([{'Model':'PMML using BYOM','Accuracy':acc, 'Precision':prec, 'F1-Score':f1}])
pmml_metrics

In [ ]:
sklearn2pmml(pipeline, "my_model.pmml", with_repr = True)

In [ ]:
additional_columns = {"Description": type("RandomForestClassifier model"),
                              "UserId": type('demo_user'),
                              "ProductionReady": False,
                              "ModelAccuracy": float(acc),
                              "ModelPrecision": prec,
                              "ModelAUC": auc,
                              "Modelf1Score": f1,
                              "ModelSavedTime": str(datetime.now(tz=pytz.UTC)),
                              "ModelGeneratedTime": end-start,
                              "sklearnVersion": sklearn.__version__
                             }
for k in additional_columns.keys():
    print(type(additional_columns[k]))

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.2 Save the model file</b></p>

In [ ]:
try:
    save_byom(model_id = 'model_anomaly1',
          model_file = 'my_model.pmml',
          table_name = 'BYOM_PMMLMODELS_REPOSITORY',
          additional_columns={"Description": "RandomForestClassifier model",
                              "UserId": 'demo_user',
                              "ProductionReady": False,
                              "ModelAccuracy": float(acc),
                              "ModelPrecision": float(prec),
                              "ModelAUC": float(auc),
                              "Modelf1Score": float(f1),
                              "ModelSavedTime": str(datetime.now(tz=pytz.UTC)),
                              "ModelGeneratedTime": float(end-start),
                              "sklearnVersion": sklearn.__version__
                             }
            )
except Exception as e: 
    # if our model exists, delete and rewrite 
    if str(e.args).find('TDML_2200') >= 1: 
        delete_byom(model_id = 'model_anomaly1', table_name = 'BYOM_PMMLMODELS_REPOSITORY') 
        save_byom(model_id = 'model_anomaly1',
              model_file = 'my_model.pmml',
              table_name = 'BYOM_PMMLMODELS_REPOSITORY',
              additional_columns={"Description": "RandomForestClassifier model",
                              "UserId": 'demo_user',
                              "ProductionReady": False,
                              "ModelAccuracy": float(acc),
                              "ModelPrecision": float(prec),
                              "ModelAUC": float(auc),
                              "Modelf1Score": float(f1),
                              "ModelSavedTime": str(datetime.now(tz=pytz.UTC)),
                              "ModelGeneratedTime": float(end-start),
                              "sklearnVersion": sklearn.__version__
                             }
            )
    else:    
        raise ValueError(f"Unable to save the model due to the following error: {e}")
#     pass 
# else: 
#     raise    



<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The model file is saved as can be found in the left navigation pane in /UseCases/Anomaly_Detection.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We create new scaled data to apply this model and predict data. New dataset is created by joining the features and the anomalies.</p>

In [ ]:
newdata = features[['WELDING_ID']+feature_names].join(other=anomalies, how='inner', on='WELDING_ID=WELDING_ID',rsuffix='r',lsuffix='l')
newdata = newdata.assign(WELDING_ID=newdata.l_WELDING_ID).drop(['l_WELDING_ID','r_WELDING_ID'],axis=1).select(['WELDING_ID']+feature_names+['anomaly'])
newdata

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We create new transformed data by using the same Scalefit object we used earlier and get the transformed data for this new data.</p>

In [ ]:
newdata_scaled = ScaleTransform(data=newdata,
                         object=scaler.output,
                                # DataFrame(in_schema('demo_user','scaler_anomaly')),
                         accumulate=["WELDING_ID","anomaly"]).result
newdata_scaled

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>6.3 Retrieve the model file and use it to predict</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We use the PMMLPredict function from the teradataml library to predict the anomalies.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Predictive Model Markup Language (PMML) is an XML-based standard established by the Data Mining Group (DMG) for defining statistical and data-mining models. PMML models can be shared between PMML-compliant platforms and across organizations so that business analysts and developers are unified in designing, analyzing, and implementing PMML-based assets and services.</p>

In [ ]:
from teradataml import PMMLPredict
modeldata_anomaly = retrieve_byom("model_anomaly1", table_name="BYOM_PMMLMODELS_REPOSITORY")
result=PMMLPredict(
                modeldata = modeldata_anomaly,
                newdata = newdata_scaled,
                accumulate = ['WELDING_ID'],
                model_output_fields=['probability(0)','probability(1)'],
                overwrite_cached_models = '*'
                )
pmml_predict=result.result
pmml_predict

In [ ]:
pmml_predict_result = pmml_predict.join(other=newdata_scaled, how='inner', on='WELDING_ID=WELDING_ID',rsuffix='r',lsuffix='l')
pmml_predict_result = pmml_predict_result.assign(prob_0=pmml_predict_result['probability(0)'])
pmml_predict_result = pmml_predict_result.assign(prob_1=pmml_predict_result['probability(1)'])
pmml_predict_result = pmml_predict_result.assign(WELDING_ID=pmml_predict_result.l_WELDING_ID)
pmml_predict_result = pmml_predict_result.assign(prediction=case([(pmml_predict_result.prob_1>pmml_predict_result.prob_0, 1 )],else_ = 0))
pmml_predict_result = pmml_predict_result.select(['WELDING_ID']+['anomaly']+['prob_0']+['prob_1']+['prediction'])
pmml_predict_result

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>7. Decision Forest </b></p>
 
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We will now use the DecisionForest model to predict the anomalies. A decision forest is a generic term to describe models made of multiple decision trees. The prediction of a decision forest is the aggregation of the predictions of its decision trees. The implementation of this aggregation depends on the algorithm used to train the decision forest. The goal of using a Decision Tree is to create a training model that can use to predict the class or value of the target variable by learning simple decision rules inferred from prior data(training data).</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We start by creating a subset for the most interesting part lies between 40 and 400ms from the start of the curve.</p>
        
        

In [ ]:
DF_curves_zoom = welding_dataset_new[(welding_dataset_new.TIME_MS > 40) & (welding_dataset_new.TIME_MS < 400) ]
DF_curves_zoom

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We create various features by using the window function on the Resistance and taking the difference between the previous and current resistance based on time. We will create these features by using the aggregation function on this resistance and the difference of the resistance.</p>
        

In [ ]:
DF_curves_zoom = DF_curves_zoom.assign(
    resistance_diff = DF_curves_zoom.RESISTANCE 
                        - DF_curves_zoom.RESISTANCE.window(
                                partition_columns=['WELDING_ID'],
                                order_columns=["TIME_MS"]
                            ).lag(1)
)
# DF_curves_zoom[DF_curves_zoom.WELDING_ID==138].sort("TIME_MS")

In [ ]:
DF_features = DF_curves_zoom.groupby("WELDING_ID").agg({
    'RESISTANCE':['sum', 'min', 'max', 'mean', 'std', 'var','skew','kurtosis'],
    'resistance_diff':['min']
})
DF_features

In [ ]:
feature_names = DF_features.columns[1:]
feature_names

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>7.1 Build the analytical dataset.</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We create the analytical dataset joining the anomaly table created above and the dataset with the features created.</p>

In [ ]:
DF_target = DataFrame('Anomoly_Target')

In [ ]:
DF_ADS_train = DF_features[['WELDING_ID']+feature_names].join(
    other=DF_target, how='inner', on='WELDING_ID=WELDING_ID',rsuffix='r',lsuffix='l')
DF_ADS_train = DF_ADS_train.assign(WELDING_ID=DF_ADS_train.l_WELDING_ID
                                  ).drop(['l_WELDING_ID','r_WELDING_ID'],axis=1
                                        ).select(['WELDING_ID']+feature_names+['anomaly']
                                                ).assign(anomaly_int = DF_ADS_train.anomaly.cast(INTEGER()))
DF_ADS_train

In [ ]:
DF_ADS_score = DF_features[['WELDING_ID']+feature_names]\
                            [DF_features.WELDING_ID>800]
DF_ADS_score

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We store these training and scoring datasets into Vantage to be used by the In-DB functions.</p>

In [ ]:
DF_ADS_train.to_sql(
    table_name  = 'ADS_train_data',
    primary_index= 'WELDING_ID',
    if_exists = 'replace'
)

In [ ]:
DF_ADS_score.to_sql(
    table_name  = 'ADS_test_data',
    primary_index= 'WELDING_ID',
    if_exists = 'replace'
)

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>7.2 Train Decision Forest</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/search/all?query=TD_DecisionForest&content-lang=en-US'>TD_DecisionForest</a> is an ensemble algorithm used for classification and regression predictive modelling problems. It is an extension of bootstrap aggregation (bagging) of decision trees. </p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function takes the training data as input, as well as the following function parameters</p>
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>InputColumns; list or range of columns used as features (we used an ordinal reference of columns 2:217)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>ResponseColumn; the dependent or target value (we used “class”, the first column)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>TreeType; either CLASSIFICATION or REGRESSION</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Other hyperparameter values detailed in the documentation</li>
        </ul>

In [ ]:
%%capture
query = '''Create table DF_train as (
SELECT * FROM TD_DecisionForest (
ON ADS_train_data AS INPUTTABLE partition by ANY
USING
  ResponseColumn('anomaly_int')
InputColumns('sum_RESISTANCE', 'min_RESISTANCE', 'max_RESISTANCE', 'mean_RESISTANCE', 'std_RESISTANCE', 'var_RESISTANCE', 'skew_RESISTANCE',
 'kurtosis_RESISTANCE', 'min_resistance_diff')
MaxDepth(16)
MinNodeSize(1)
NumTrees(8)
ModelType('CLASSIFICATION')
Seed(3)
Mtry(1)
MtrySeed(3)
) AS dt
) with data;
'''
 
try:
    execute_sql(query)
except:
    execute_sql('DROP TABLE DF_train;')
    execute_sql(query)

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>7.3 Predict and Evaluate Decision Forest model</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute a testing prediction using the split data above.  Evaluate the model by creating a confusion matrix with the <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> SQL Function.</p>


<ol style = 'font-size:16px;font-family:Arial;color:#00233C'>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute <a href = 'https://docs.teradata.com/r/Teradata-VantageTM-Analytics-Database-Analytic-Functions-17.20/Model-Scoring-Functions/DecisionForestPredict'>DecisionForestPredict</a> using the model built above</li>
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Execute <a href = 'https://docs.teradata.com/search/all?query=TD_ClassificationEvaluator&content-lang=en-US'>TD_ClassificationEvaluator</a> and pass the actual classification and the predicted value</li>
</ol>

In [ ]:
%%capture
query = '''
Create table DF_Predict as (
SELECT * FROM TD_DecisionForestPredict (
ON ADS_train_data AS InputTable PARTITION BY ANY
ON DF_Train AS ModelTable DIMENSION
USING
  IdColumn ('WELDING_ID')
  Accumulate ('anomaly_int')
  Detailed('false')
  OutputProb ('true')
  Responses ('0','1')
) AS dt) with data;'''

try:
    execute_sql(query)
except:
    execute_sql('DROP TABLE DF_Predict;')
    execute_sql(query)

In [ ]:
df_predict= DataFrame('DF_Predict')
df_predict_char = df = df_predict.assign(anomaly = df_predict.anomaly_int.cast(type_=VARCHAR(2))
                                        ,prediction_ch = df_predict.prediction.cast(type_=VARCHAR(2)))
df_predict_char

In [ ]:
ClassificationEvaluator_obj = ClassificationEvaluator(data=df_predict_char,
                                                          observation_column='anomaly',
                                                          prediction_column='prediction_ch',
                                                          labels=['0','1'])


In [ ]:
df_metrics = ClassificationEvaluator_obj.output_data
df_metrics

In [ ]:
df_metric_pd = df_metrics.to_pandas(all_rows = True)

In [ ]:
df_metric_pd['Metric'] = df_metric_pd['Metric'].str.strip('\x00')

In [ ]:
accuracy = df_metric_pd[df_metric_pd['Metric'] == 'Accuracy']['MetricValue'][0]
precision = df_metric_pd[df_metric_pd['Metric'] == 'Micro-Precision']['MetricValue'][1]
f1score = df_metric_pd[df_metric_pd['Metric'] == 'Micro-F1']['MetricValue'][3]

In [ ]:
df_metrics_new=pd.DataFrame([{'Model':'In-DB DecisionForest','Accuracy':accuracy, 'Precision':precision, 'F1-Score':f1score}])
df_metrics_new

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>7.4 Score new Data</b></p>

In [ ]:
%%capture
query = '''
Create table DF_Predict_test as (
SELECT * FROM TD_DecisionForestPredict (
ON ADS_test_data AS InputTable PARTITION BY ANY
ON DF_Train AS ModelTable DIMENSION
USING
  IdColumn ('WELDING_ID')
  Detailed('false')
  OutputProb ('true')
  Responses ('0','1')
) AS dt) with data;'''

try:
    execute_sql(query)
except:
    execute_sql('DROP TABLE DF_Predict_test;')
    execute_sql(query)

In [ ]:
df_predict_test= DataFrame('DF_Predict_test')
df_predict_test

<hr style="height:2px;border:none;background-color:#00233C;">

<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>8. Compare PMML and DecisionForest</b></p>
<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>8.1 Show AUC-ROC Curve</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The <a href = 'https://docs.teradata.com/search/all?query=TD_ROC&content-lang=en-US'>ROC</a> curve shows the performance of a binary classification model as its discrimination threshold varies. For a range of thresholds, the curve plots the true positive rate against false-positive rate.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>This function accepts a set of prediction-actual pairs as input and calculates the following values for a range of discrimination thresholds.</p>
    <ul style = 'font-size:16px;font-family:Arial;color:#00233C'>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>True-positive rate (TPR)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>False-positive rate (FPR)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>The area under the ROC curve (AUC)</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Gini coefficient</li>
        <li style = 'font-size:16px;font-family:Arial;color:#00233C'>Other details are mentioned in the documentation</li>
    </ul>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>ROC for PMML</b></p>

In [ ]:
from teradataml import ROC 
roc_pmml = ROC(data = pmml_predict_result, 
                    probability_column = "prob_1",
                    observation_column = "anomaly",
                    positive_class="1"
                    )

In [ ]:
roc_data_pmml = roc_pmml.output_data.to_pandas().sort_values("fpr", ascending=True)
roc_data_pmml.tail(10)

In [ ]:
auc_pmml = roc_pmml.result.to_pandas().iloc[0,0]
auc_pmml

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>ROC for DecisionForest</b></p>

In [ ]:
roc_obj = ROC(data = df_predict, 
                    probability_column = "prob_1",
                    observation_column = "anomaly_int",
                    positive_class="1"
                    )

In [ ]:
roc_data = roc_obj.output_data.to_pandas().sort_values("fpr", ascending=True)
roc_data.tail(10)

In [ ]:
auc = roc_obj.result.to_pandas().iloc[0,0]
auc

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Plot ROC Curves</b></p>

In [ ]:
# Plot 1
plt.plot(roc_data_pmml['fpr'], roc_data_pmml['tpr'], color='orange', label='PMML ROC. AUC = {}'.format(str(auc_pmml)), drawstyle='steps') 
# Plot 2
plt.plot(roc_data['fpr'], roc_data['tpr'], color='green', label='DecisionForest ROC. AUC = {}'.format(str(auc)),  drawstyle='steps') 
# Plot the diagonal dashed line
plt.plot([0, 1], [0, 1], color='darkblue', linestyle='--') 
# Set labels and title
plt.xlabel('False Positive Rate',fontsize=12) 
plt.ylabel('True Positive Rate',fontsize=12) 
plt.title('Receiver Operating Characteristic (ROC) Curve',fontsize=16) 
# Add legend
plt.legend(loc="lower right",fontsize=10) 
# Show the plot
plt.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The closer the ROC curve is to the upper left corner of the graph, the higher the accuracy of the test because in the upper left corner, the sensitivity = 1 and the false positive rate = 0 (specificity = 1). The ideal ROC curve thus has an AUC = 1.0. As seen in the above graph the AUC for both the models is close to 1 so the accuracy of both models is very good. </p>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>8.2 Show Confusion Matrix</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Confusion Matrix is a performance measurement for machine learning classification problem where output can be two or more classes. It is a table with 4 different combinations of predicted and actual values.</p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Confusion matrices represent counts from predicted and actual values. The output “TN” stands for True Negative which shows the number of negative examples classified accurately. Similarly, “TP” stands for True Positive which indicates the number of positive examples classified accurately. The term “FP” shows False Positive value, i.e., the number of actual negative examples classified as positive; and “FN” means a False Negative value which is the number of actual positive examples classified as negative.</p>


In [ ]:
# Calculate confusion matrix for PMML
DF_result=df_predict.to_pandas().reset_index()
pmml_result=pmml_predict_result.to_pandas()
cm_pmml = confusion_matrix(pmml_result['anomaly'], pmml_result['prediction']) 
# Calculate confusion matrix for DecisionForest
cm_df = confusion_matrix(DF_result['anomaly_int'], DF_result['prediction']) 
# Create figure and axes objects
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8)) 
# Plot PMML confusion matrix
disp_pmml = ConfusionMatrixDisplay(confusion_matrix=cm_pmml, display_labels=['No Anomaly', 'Anomaly']) 
disp_pmml.plot(ax=ax1, cmap='Blues', colorbar=False) 
ax1.set_title('PMML Confusion Matrix') 
ax1.set_xlabel('Predicted Label') 
ax1.set_ylabel('True Label') 
ax1.set_xticks([0, 1]) 
ax1.set_yticks([0, 1]) 
ax1.set_xticklabels(['No Anomaly', 'Anomaly']) 
ax1.set_yticklabels(['No Anomaly', 'Anomaly'])

# Add text to the plot to show the actual values of the confusion matrix
for i in range(cm_pmml.shape[0]): 
    for j in range(cm_pmml.shape[1]): 
        ax1.text(j, i, f'{cm_pmml[i, j]}', ha='center', va='center', color='white' if cm_pmml[i, j] > cm_pmml.max() / 2 else 'black') 

# Plot DecisionForest confusion matrix
disp_df = ConfusionMatrixDisplay(confusion_matrix=cm_df, display_labels=['No Anomaly', 'Anomaly']) 
disp_df.plot(ax=ax2, cmap='Blues', colorbar=False) 
ax2.set_title('DecisionForest Confusion Matrix') 
ax2.set_xlabel('Predicted Label') 
ax2.set_ylabel('True Label') 
ax2.set_xticks([0, 1]) 
ax2.set_yticks([0, 1]) 
ax2.set_xticklabels(['No Anomaly', 'Anomaly']) 
ax2.set_yticklabels(['No Anomaly', 'Anomaly'])

# Add text to the plot to show the actual values of the confusion matrix
for i in range(cm_df.shape[0]): 
    for j in range(cm_df.shape[1]): 
        ax2.text(j, i, f'{cm_df[i, j]}', ha='center', va='center', color='white' if cm_df[i, j] > cm_df.max() / 2 else 'black') 

# Adjust layout and spacing
plt.tight_layout() 
# Show the plot
plt.show()

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The confusion matrix for this binary class classification problem has the below 4 quadrants: </p>

<li style = 'font-size:16px;font-family:Arial;color:#00233C'>True Positive (TP) refers to a sample belonging to the positive class being classified correctly.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>True Negative (TN) refers to a sample belonging to the negative class being classified correctly.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>False Positive (FP) refers to a sample belonging to the negative class but being classified wrongly as belonging to the positive class.</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'>False Negative (FN) refers to a sample belonging to the positive class but being classified wrongly as belonging to the negative class.</li>

<hr style="height:1px;border:none;background-color:#00233C;">
<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>8.3 Show Metrices</b></p>

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Below is the comparison for Accuracy, Precision and F1-Score of the 2 models.</p>
<table style = 'font-size:16px;font-family:Arial;color:#00233C'>
  <tr>
    <th>Column</th>
    <th>Description</th>
  </tr>
  <tr>
    <td>Precision</td>
    <td>The positive predictive value. Refers to the fraction of relevant instances among
the total retrieved instances.
        Precision answers the following question: what proportion of predicted Positives is truly Positive? 
        Precision = (TP)/(TP+FP)</td>
  </tr>
  <tr>
    <td>Accuracy</td>
    <td>Accuracy simply measures how often the classifier correctly predicts. We can define accuracy as the ratio of the number of correct predictions and the total number of predictions.</td>
  </tr>
  <tr>
    <td>F1</td>
    <td>F1 score, defined as the harmonic mean of the precision and recall and is a number between 0 and 1. F1 score maintains a balance between the precision and recall for your classifier.                                         
                      F1 = 2*(precision*recall/precision+recall)</td>
  </tr>
</table>


In [ ]:
combined_metrics=pd.concat([pmml_metrics, df_metrics_new], axis=0)
combined_metrics

<p style = 'font-size:16px;font-family:Arial;color:#00233C'>From the above metrics we can conclude that both the models are performing almost similar and have similar Accuracy and Precision.</p>

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b> Conclusion</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>We have seen an end-to-end exploration process for labelling anomalous time series using ClearScape Analytics on Teradata Vantage. Thanks to the in-database capabilities offered by Teradata Vantage with ClearScape Analytics, we were able to run this exploration with the smallest notebook instance. The unique massively-parallel architecture of Teradata Vantage allows users to prepare data, train, evaluate, and deploy models at unprecedented scale.</p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>In this particular use case, we have observed that with large volume of machine sensor data millions of ML models were created to derive analytic features that ultimately deployed tens of thousands of models for real-time scoring. This extent of scale is only possible by combining the power of Vantage with native ClearScape Analytic functions.</p>

<hr style="height:2px;border:none;background-color:#00233C;">
<p style = 'font-size:20px;font-family:Arial;color:#00233C'><b>8. Cleanup</b></p>
<p style = 'font-size:18px;font-family:Arial;color:##00233C'><b>Work Tables</b></p>

In [ ]:
tables = ['ADS_train_data', 'ADS_test_data','DF_train', 'DF_Predict', 'DF_Predict_test','additional_metrics_test']

# Loop through the list of tables and execute the drop table command for each table
for table in tables:
    try:
        db_drop_table(table_name=table)
    except:
        pass
    # Construct the drop table SQL statement
    # drop_table_sql = f"DROP TABLE {table};"
    # Execute the drop table command
    # eng.execute(drop_table_sql)

<p style = 'font-size:18px;font-family:Arial;color:#00233C'><b>Databases and Tables</b></p>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>The following code will clean up tables and databases created above.</p>

In [ ]:
%run -i ../run_procedure.py "call remove_data('DEMO_AnomalyDetection');" 
#Takes 40 seconds

In [ ]:
remove_context()

<hr style="height:2px;border:none;background-color:#00233C;">
<b style = 'font-size:20px;font-family:Arial;color:#00233C'>Required Materials</b>
<p style = 'font-size:16px;font-family:Arial;color:#00233C'>Let’s look at the elements we have available for reference for this notebook:</p>
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Filters:</b> 
    <li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Industry:</b> Manufacturing</li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Functionality:</b> Machine Learning</li> 
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><b>Use Case:</b> Anomaly Detection</li></p>
<b style = 'font-size:18px;font-family:Arial;color:#00233C'>Related Resources:</b>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://www.teradata.com/Blogs/Hyper-scale-time-series-forecasting-done-right'>Hyper-scale time series forecasting done right</a> </li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://www.teradata.com/Resources/Datasheets/Stay-Ahead-of-Rapid-Change-with-a-Dynamic-Supply-Chain?utm_campaign=i_coremedia-AMS&utm_source=google&utm_medium=paidsearch&utm_content=GS_CoreMedia_NA-US_BKW&utm_creative=Brand-Vantage&utm_term=teradata%20analytic%20platform&gclid=Cj0KCQjwnMWkBhDLARIsAHBOftrWZxDktHkKMsaWjMmNRnQ6Ys-bZBAUhXjWTo1Xa02fsci-IHWBV_waAppkEALw_wcB'>Stay Ahead of Continuous and Rapid Change with a Dynamic Supply Chain</a></li>
<li style = 'font-size:16px;font-family:Arial;color:#00233C'><a href = 'https://www.teradata.com/Industries/Manufacturing'>Achieve industry 4.0 using advanced manufacturing analytics at scale</a></li>



<footer style="padding-bottom:35px; background:#f9f9f9; border-bottom:3px solid #00233C">
    <div style="float:left;margin-top:14px">ClearScape Analytics™</div>
    <div style="float:right;">
        <div style="float:left; margin-top:14px">
            © 2023, 2024 Teradata. All rights reserved.
        </div>
    </div>
</footer>